In [88]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import matplotlib.pyplot as plt
import random as rnd
import pandas as pd
import numpy as np
import csv

%matplotlib inline

# Datos RegLinPoli2

In [89]:
Location = r'C:\Users\ThinkYoga\Desktop\ml17\reglin\regLinPoli2.csv'
df = pd.read_csv(Location)

In [90]:
# Separate both train and test as well as the response variable
index=np.array([1 if np.random.random() < 0.75 else 0 for i in range(len(df))])
X_train=np.array(df[df.columns[0:-1]])[index==1]
Y_train=np.array(df[df.columns[-1]])[index==1]
X_test=np.array(df[df.columns[0:-1]])[index==0]
Y_test=np.array(df[df.columns[-1]])[index==0]
train=np.array(df)[index==1]

In [91]:
#Normalizamos X
scaler = preprocessing.StandardScaler().fit(train)
train = scaler.transform(train)

In [92]:
def RegLinLMSmatReg(X,Y,W,eta,lam):
    for i in range(len(X)):
        Vxi = np.dot(W,np.insert(X_train[i],0,1))
        error = Y[i] - Vxi
        W = W + (eta * error) * np.insert(X[i],0,1) - lam*W

    return W

In [93]:
## Transfer function
def salida(w,X):
    return X.dot(w[1:]) +w[0]

In [94]:
## Training function
def entrena(X,y,w,la=0.0,eta=0.01):
    
    for i in range(len(X)):
        error=y[i]-salida(w,X[i])
        w[0]=w[0]+eta*(error)
        w[1:]=w[1:]+eta*(error*X[i])-la*w[1:]
    return w

In [95]:
def calcError(X,y,w,w0):
    return np.mean((X.dot(w)+w0-y)**2)

In [111]:
def CrossValLambda(dfsplitted,testlam):
    ECMmatrix = np.zeros((len(dfsplitted),len(testlam)))
    for i in range(len(dfsplitted)):
        testtemp=dfsplitted[i]
        traintemp=np.delete(dfsplitted,i)
        
        #Juntamos los subconjuntos sobrantes para formar el training set temporal
        traintempf = traintemp[0]
        for j in range(1,len(traintemp)):
            traintempf=np.concatenate((traintempf,traintemp[j]))

        # Separamos el conjunto de entrenamiento y prueba temporales en X y Y
        X_traintemp = traintempf[:,range(len(traintempf[0])-1)]
        Y_traintemp = traintempf[:,len(traintempf[0])-1]
        X_testtemp = testtemp[:,range(len(traintempf[0])-1)]
        Y_testtemp = testtemp[:,len(traintempf[0])-1]
        
        for k in range(len(testlam)):
            #W = np.ones(len(X_traintemp[0])+1)
            #for l in range(100):
            #    W = RegLinLMSmatReg(X_traintemp,Y_traintemp,W,0.1,testlam[k])
            W=np.asarray([rnd.random() for m in range(1+len(X_traintemp[0]))])
            for l in range(100):
                W=entrena(X_traintemp,Y_traintemp,W,testlam[k])

            ECMmatrix[i][k] = calcError(X_testtemp,Y_testtemp,W[1:],W[0])

    
    return ECMmatrix

In [97]:
np.random.seed(42)
np.random.shuffle(train)
train_split = np.array_split(train,10)

In [114]:
# Regresamos una matriz con los errores en cada iteración para los respectivos valores de lambda
ECMmatrixLambdas = CrossValLambda(train_split,[0.0,0.05,0.1,0.2])
ECMmatrixLambdas

array([[ 0.58864966,  0.98343455,  1.026029  ,  1.04959315],
       [ 0.64093592,  1.09425836,  1.1690782 ,  1.2270241 ],
       [ 0.43498941,  0.54073663,  0.5685176 ,  0.587     ],
       [ 0.51101543,  0.95008006,  1.00366932,  1.04191338],
       [ 0.7171659 ,  1.04562205,  1.0889841 ,  1.12053365],
       [ 0.54988222,  0.92558056,  0.96885504,  0.99968879],
       [ 0.61378143,  0.92956678,  0.96085985,  0.98395922],
       [ 0.54314342,  0.85935545,  0.89134363,  0.91254926],
       [ 0.53849865,  0.88117403,  0.93943404,  0.98430926],
       [ 0.54417557,  0.46381934,  0.46548477,  0.46974189]])

## Vemos que la media de los errores para lambda = 0 es la menor

In [115]:
ECMmatrixLambdas.mean(0)

array([ 0.56822376,  0.86736278,  0.90822555,  0.93763127])